# Week 6 - Interactive Visualizations Part 2: Dashboard Layouts

## Learning Objectives
By the end of this lesson, you will be able to:
1. Design professional dashboard layouts using Plotly subplots
2. Create coordinated visualizations that tell cohesive business stories
3. Implement advanced layout techniques for executive presentations
4. Build responsive dashboards suitable for business intelligence

## Business Context: The Power of Dashboards

In today's data-driven business environment, executives and stakeholders need **at-a-glance insights** that tell complete stories. A well-designed dashboard serves multiple critical functions:

- **Executive Decision Making** - Quick overview of key metrics and trends
- **Performance Monitoring** - Real-time tracking of business KPIs
- **Stakeholder Communication** - Visual storytelling for diverse audiences
- **Operational Efficiency** - Identifying bottlenecks and opportunities

Today we'll learn to create professional dashboard layouts using real Olist e-commerce data that provide comprehensive business intelligence.

## 1. Dashboard Design Principles

Before diving into code, let's establish key principles for effective dashboard design:

### Visual Hierarchy
- **Most important metrics** → Top-left (natural reading pattern)
- **Supporting details** → Lower sections
- **Context information** → Right side or bottom

### Information Density
- **Executive dashboards** → High-level KPIs, minimal detail
- **Operational dashboards** → Detailed metrics, drill-down capability
- **Analytical dashboards** → Complex relationships, multiple perspectives

### Color Strategy
- **Consistent palette** → Professional appearance
- **Semantic colors** → Red for alerts, green for positive trends
- **Accessibility** → Colorblind-friendly combinations

In [10]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import zipfile
import io
import requests
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Professional color palette for business dashboards
BUSINESS_COLORS = {
    'primary': '#1f77b4',      # Professional blue
    'secondary': '#ff7f0e',    # Corporate orange
    'success': '#2ca02c',      # Success green
    'warning': '#d62728',      # Alert red
    'info': '#9467bd',         # Information purple
    'neutral': '#8c564b',      # Neutral brown
    'light': '#e7e7e7',       # Light gray
    'dark': '#2f2f2f'         # Dark gray
}

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("Dashboard development environment ready! 📊")
print("Color palette loaded for professional business visualizations")

Dashboard development environment ready! 📊
Color palette loaded for professional business visualizations


In [11]:
# Load Olist data (using the same function from Part 1)
def load_olist_data_from_zip():
    """
    Load Olist e-commerce data directly from the GitHub repository zip file.
    """
    try:
        zip_url = "https://github.com/autom8or-com/python-data-analysis-course/raw/main/Resources/data/sales.zip"
        
        print("Loading Olist dataset for dashboard creation...")
        response = requests.get(zip_url)
        response.raise_for_status()
        
        datasets = {}
        
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            file_mapping = {
                'customers': 'olist_customers_dataset.csv',
                'orders': 'olist_orders_dataset.csv',
                'order_items': 'olist_order_items_dataset.csv',
                'products': 'olist_products_dataset.csv',
                'sellers': 'olist_sellers_dataset.csv',
                'payments': 'olist_order_payments_dataset.csv',
                'reviews': 'olist_order_reviews_dataset.csv'
            }
            
            for table_name, filename in file_mapping.items():
                try:
                    datasets[table_name] = pd.read_csv(zip_file.open(filename))
                    print(f"✓ {table_name}: {datasets[table_name].shape[0]:,} rows")
                except KeyError:
                    continue
        
        return datasets
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return create_sample_dashboard_data()

def create_sample_dashboard_data():
    """
    Create comprehensive sample data optimized for dashboard demonstration.
    """
    np.random.seed(42)
    
    # Enhanced sample data with business metrics
    n_customers = 8000
    n_orders = 15000
    n_items = 25000
    
    # Brazilian states with realistic distribution
    states = ['SP', 'RJ', 'MG', 'RS', 'PR', 'SC', 'BA', 'GO', 'PE', 'CE', 'DF', 'ES']
    state_weights = [0.35, 0.15, 0.12, 0.08, 0.06, 0.05, 0.04, 0.03, 0.03, 0.03, 0.03, 0.03]
    
    # Product categories with business focus
    categories = [
        'health_beauty', 'computers_accessories', 'furniture_decor', 'sports_leisure',
        'housewares', 'watches_gifts', 'telephony', 'auto', 'toys', 'cool_stuff',
        'garden_tools', 'perfumery', 'baby', 'electronics', 'stationery'
    ]
    
    # Generate time-based data with seasonal patterns
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2018, 8, 31)
    
    # Create datasets
    customers = pd.DataFrame({
        'customer_id': [f'cust_{i:06d}' for i in range(n_customers)],
        'customer_state': np.random.choice(states, n_customers, p=state_weights),
        'customer_city': np.random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 
                                         'Porto Alegre', 'Curitiba', 'Salvador', 'Brasília'], n_customers)
    })
    
    # Orders with seasonal trends
    order_dates = pd.date_range(start_date, end_date, periods=n_orders)
    orders = pd.DataFrame({
        'order_id': [f'ord_{i:08d}' for i in range(n_orders)],
        'customer_id': np.random.choice(customers['customer_id'], n_orders),
        'order_status': np.random.choice(['delivered', 'shipped', 'processing', 'canceled'], 
                                       n_orders, p=[0.87, 0.07, 0.04, 0.02]),
        'order_purchase_timestamp': order_dates,
        'order_delivered_customer_date': order_dates + pd.to_timedelta(
            np.random.exponential(7, n_orders), unit='days'
        )
    })
    
    # Order items with realistic pricing
    order_items = pd.DataFrame({
        'order_id': np.random.choice(orders['order_id'], n_items),
        'product_id': [f'prod_{i:06d}' for i in np.random.randint(0, 3000, n_items)],
        'seller_id': [f'sell_{i:04d}' for i in np.random.randint(0, 800, n_items)],
        'price': np.random.lognormal(3.2, 0.9, n_items),
        'freight_value': np.random.exponential(12, n_items)
    })
    
    # Products with categories
    products = pd.DataFrame({
        'product_id': [f'prod_{i:06d}' for i in range(3000)],
        'product_category_name': np.random.choice(categories, 3000),
        'product_weight_g': np.random.lognormal(6.5, 1.2, 3000)
    })
    
    # Customer reviews for satisfaction metrics
    reviews = pd.DataFrame({
        'order_id': np.random.choice(orders['order_id'], int(n_orders * 0.8)),
        'review_score': np.random.choice([1, 2, 3, 4, 5], int(n_orders * 0.8), 
                                       p=[0.05, 0.08, 0.15, 0.32, 0.40])
    })
    
    print("✓ Enhanced sample datasets created for dashboard demonstration")
    
    return {
        'customers': customers,
        'orders': orders,
        'order_items': order_items,
        'products': products,
        'reviews': reviews
    }

# Load the data
data = load_olist_data_from_zip()
print("\n🚀 Dashboard data ready!")

Loading Olist dataset for dashboard creation...
✓ customers: 99,441 rows
✓ orders: 99,441 rows
✓ order_items: 112,650 rows
✓ products: 32,951 rows
✓ sellers: 3,095 rows
✓ payments: 103,886 rows
✓ reviews: 99,224 rows

🚀 Dashboard data ready!


In [12]:
def prepare_dashboard_metrics(data):
    """
    Prepare comprehensive business metrics for dashboard visualization.
    """
    print("Calculating comprehensive business metrics...")
    
    # Create master dataset
    df = data['orders'].copy()
    
    # Convert dates
    df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], errors='coerce')
    df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'], errors='coerce')
    
    # Add customer data
    df = df.merge(data['customers'], on='customer_id', how='left')
    
    # Calculate order financials
    order_financials = data['order_items'].groupby('order_id').agg({
        'price': 'sum',
        'freight_value': 'sum',
        'product_id': 'count'
    }).reset_index()
    order_financials.columns = ['order_id', 'order_value', 'freight_value', 'items_count']
    order_financials['total_value'] = order_financials['order_value'] + order_financials['freight_value']
    
    df = df.merge(order_financials, on='order_id', how='left')
    
    # Add product categories
    if 'products' in data:
        order_categories = data['order_items'].merge(
            data['products'][['product_id', 'product_category_name']], 
            on='product_id', how='left'
        )
        primary_categories = order_categories.loc[
            order_categories.groupby('order_id')['price'].idxmax()
        ][['order_id', 'product_category_name']]
        
        df = df.merge(primary_categories, on='order_id', how='left')
    
    # Add review scores
    if 'reviews' in data:
        avg_reviews = data['reviews'].groupby('order_id')['review_score'].mean().reset_index()
        df = df.merge(avg_reviews, on='order_id', how='left')
    
    # Create time features
    df['year'] = df['order_purchase_timestamp'].dt.year
    df['month'] = df['order_purchase_timestamp'].dt.month
    df['quarter'] = df['order_purchase_timestamp'].dt.quarter
    df['weekday'] = df['order_purchase_timestamp'].dt.day_name()
    df['year_month'] = df['order_purchase_timestamp'].dt.to_period('M')
    
    # Calculate delivery metrics
    df['delivery_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
    
    # Clean data
    df = df.fillna({
        'order_value': 0,
        'freight_value': 0,
        'total_value': 0,
        'items_count': 1,
        'review_score': 3.5,
        'product_category_name': 'uncategorized',
        'delivery_days': 10
    })
    
    print(f"✓ Dashboard metrics prepared: {df.shape[0]:,} orders")
    return df

# Prepare comprehensive metrics
dashboard_df = prepare_dashboard_metrics(data)

# Key metrics summary
total_revenue = dashboard_df['total_value'].sum()
total_orders = len(dashboard_df)
avg_order_value = dashboard_df['total_value'].mean()
customer_satisfaction = dashboard_df['review_score'].mean()

print(f"\n📊 Key Business Metrics:")
print(f"  💰 Total Revenue: ${total_revenue:,.0f}")
print(f"  📦 Total Orders: {total_orders:,}")
print(f"  💵 Average Order Value: ${avg_order_value:.2f}")
print(f"  ⭐ Customer Satisfaction: {customer_satisfaction:.2f}/5.0")

Calculating comprehensive business metrics...
✓ Dashboard metrics prepared: 99,441 orders

📊 Key Business Metrics:
  💰 Total Revenue: $15,843,553
  📦 Total Orders: 99,441
  💵 Average Order Value: $159.33
  ⭐ Customer Satisfaction: 4.08/5.0


## 2. Executive Dashboard - High-Level KPIs

Executive dashboards focus on the **big picture**. They provide senior leadership with essential metrics at a glance, without overwhelming detail.

### Key Characteristics:
- **Large, clear numbers** for primary KPIs
- **Trend indicators** (up/down arrows, color coding)
- **Minimal detail** - focus on outcomes, not process
- **Strategic timeframes** - monthly, quarterly, yearly views

In [13]:
def create_executive_dashboard(df):
    """
    Create an executive-level dashboard with high-level KPIs and trends.
    """
    # Prepare monthly data for trends
    monthly_metrics = df.groupby('year_month').agg({
        'total_value': 'sum',
        'order_id': 'count',
        'customer_id': 'nunique',
        'review_score': 'mean',
        'delivery_days': 'mean'
    }).reset_index()
    
    monthly_metrics['date'] = monthly_metrics['year_month'].dt.to_timestamp()
    monthly_metrics['aov'] = monthly_metrics['total_value'] / monthly_metrics['order_id']
    
    # Calculate growth rates
    monthly_metrics['revenue_growth'] = monthly_metrics['total_value'].pct_change() * 100
    monthly_metrics['order_growth'] = monthly_metrics['order_id'].pct_change() * 100
    
    # Create dashboard with 2x3 layout - specify subplot types correctly
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=(
            '💰 Monthly Revenue Trend',
            '📦 Order Volume Growth',
            '⭐ Customer Satisfaction',
            '👥 Active Customers',
            '🚚 Delivery Performance',
            '📈 Average Order Value'
        ),
        specs=[
            [{"secondary_y": True}, {"secondary_y": True}, {"type": "indicator"}],
            [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": True}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.08
    )
    
    # Chart 1: Revenue with growth rate
    fig.add_trace(
        go.Scatter(
            x=monthly_metrics['date'],
            y=monthly_metrics['total_value'],
            mode='lines+markers',
            name='Revenue',
            line=dict(color=BUSINESS_COLORS['primary'], width=3),
            marker=dict(size=8),
            hovertemplate='Date: %{x}<br>Revenue: $%{y:,.0f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=monthly_metrics['date'],
            y=monthly_metrics['revenue_growth'],
            mode='lines',
            name='Growth %',
            line=dict(color=BUSINESS_COLORS['success'], width=2, dash='dot'),
            yaxis='y2',
            hovertemplate='Growth: %{y:.1f}%<extra></extra>'
        ),
        row=1, col=1, secondary_y=True
    )
    
    # Chart 2: Order volume with growth
    fig.add_trace(
        go.Bar(
            x=monthly_metrics['date'],
            y=monthly_metrics['order_id'],
            name='Orders',
            marker_color=BUSINESS_COLORS['secondary'],
            opacity=0.8,
            hovertemplate='Date: %{x}<br>Orders: %{y:,}<extra></extra>'
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=monthly_metrics['date'],
            y=monthly_metrics['order_growth'],
            mode='lines+markers',
            name='Order Growth %',
            line=dict(color=BUSINESS_COLORS['warning'], width=2),
            yaxis='y4',
            hovertemplate='Growth: %{y:.1f}%<extra></extra>'
        ),
        row=1, col=2, secondary_y=True
    )
    
    # Chart 3: Customer satisfaction gauge
    avg_satisfaction = monthly_metrics['review_score'].mean()
    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=avg_satisfaction,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "Rating (1-5)"},
            delta={'reference': 4.0},
            gauge={
                'axis': {'range': [None, 5]},
                'bar': {'color': BUSINESS_COLORS['primary']},
                'steps': [
                    {'range': [0, 2], 'color': BUSINESS_COLORS['warning']},
                    {'range': [2, 3.5], 'color': BUSINESS_COLORS['secondary']},
                    {'range': [3.5, 5], 'color': BUSINESS_COLORS['success']}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 4.0
                }
            }
        ),
        row=1, col=3
    )
    
    # Chart 4: Active customers
    fig.add_trace(
        go.Scatter(
            x=monthly_metrics['date'],
            y=monthly_metrics['customer_id'],
            mode='lines+markers',
            name='Active Customers',
            line=dict(color=BUSINESS_COLORS['info'], width=3),
            fill='tonexty',
            hovertemplate='Date: %{x}<br>Customers: %{y:,}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Chart 5: Delivery performance
    fig.add_trace(
        go.Box(
            y=df['delivery_days'],
            name='Delivery Days',
            marker_color=BUSINESS_COLORS['neutral'],
            boxpoints='outliers'
        ),
        row=2, col=2
    )
    
    # Chart 6: AOV trend with benchmark
    aov_benchmark = monthly_metrics['aov'].quantile(0.75)
    
    fig.add_trace(
        go.Scatter(
            x=monthly_metrics['date'],
            y=monthly_metrics['aov'],
            mode='lines+markers',
            name='AOV',
            line=dict(color=BUSINESS_COLORS['primary'], width=3),
            hovertemplate='Date: %{x}<br>AOV: $%{y:.2f}<extra></extra>'
        ),
        row=2, col=3
    )
    
    fig.add_hline(
        y=aov_benchmark,
        line_dash="dash",
        line_color=BUSINESS_COLORS['success'],
        annotation_text=f"Target: ${aov_benchmark:.2f}",
        row=2, col=3
    )
    
    # Update layout for executive dashboard
    fig.update_layout(
        title={
            'text': '🏢 OLIST E-COMMERCE - EXECUTIVE DASHBOARD',
            'font': {'size': 24, 'color': BUSINESS_COLORS['dark']},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=800,
        width=1400,
        showlegend=False,
        font=dict(size=12),
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Format axes
    fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=1)
    fig.update_yaxes(title_text="Growth %", tickformat='.1f', row=1, col=1, secondary_y=True)
    
    fig.update_yaxes(title_text="Orders", tickformat=',', row=1, col=2)
    fig.update_yaxes(title_text="Growth %", tickformat='.1f', row=1, col=2, secondary_y=True)
    
    fig.update_yaxes(title_text="Customers", tickformat=',', row=2, col=1)
    fig.update_yaxes(title_text="Days", row=2, col=2)
    fig.update_yaxes(title_text="AOV ($)", tickformat='$.2f', row=2, col=3)
    
    return fig

# Create and display executive dashboard
exec_dashboard = create_executive_dashboard(dashboard_df)
exec_dashboard.show()

print("\n👔 Executive Dashboard Features:")
print("  • High-level KPIs at a glance")
print("  • Growth indicators and trends")
print("  • Performance against benchmarks")
print("  • Clean, professional layout for C-suite presentations")

PlotlyKeyError: Invalid property specified for object of type plotly.graph_objs.Indicator: 'xaxis'

Did you mean "align"?

    Valid properties:
        align
            Sets the horizontal alignment of the `text` within the
            box. Note that this attribute has no effect if an
            angular gauge is displayed: in this case, it is always
            centered
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        delta
            :class:`plotly.graph_objects.indicator.Delta` instance
            or dict with compatible properties
        domain
            :class:`plotly.graph_objects.indicator.Domain` instance
            or dict with compatible propertie

## 3. Operational Dashboard - Detailed Metrics

Operational dashboards provide the **detailed insights** that managers and analysts need for day-to-day operations. They include more granular data and allow for deeper exploration.

### Key Characteristics:
- **Detailed breakdowns** by category, region, time period
- **Comparative analysis** between different segments
- **Operational metrics** - inventory, logistics, customer service
- **Actionable insights** for immediate decision-making

In [ ]:
def create_operational_dashboard(df):
    """
    Create an operational dashboard with detailed business metrics.
    """
    # Prepare data for operational metrics
    category_performance = df.groupby('product_category_name').agg({
        'total_value': ['sum', 'mean'],
        'order_id': 'count',
        'review_score': 'mean',
        'delivery_days': 'mean'
    }).round(2)
    
    category_performance.columns = ['total_revenue', 'avg_order_value', 'order_count', 'avg_rating', 'avg_delivery']
    category_performance = category_performance.reset_index().sort_values('total_revenue', ascending=False).head(12)
    
    # State performance
    state_performance = df.groupby('customer_state').agg({
        'total_value': 'sum',
        'order_id': 'count',
        'freight_value': 'mean',
        'delivery_days': 'mean'
    }).round(2).reset_index().sort_values('total_value', ascending=False).head(10)
    
    # Weekly patterns
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly_pattern = df.groupby('weekday').agg({
        'order_id': 'count',
        'total_value': 'sum'
    }).reindex(weekday_order).reset_index()
    
    # Create 3x2 operational dashboard with correct subplot types
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            '🛍️ Category Performance (Revenue vs Rating)',
            '🗺️ Geographic Distribution (Top States)',
            '📊 Category Revenue Breakdown',
            '🚚 Delivery Performance by State',
            '📅 Weekly Order Patterns',
            '💰 Revenue Distribution'
        ),
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"type": "pie"}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ],
        vertical_spacing=0.08,
        horizontal_spacing=0.1
    )
    
    # Chart 1: Category performance scatter
    fig.add_trace(
        go.Scatter(
            x=category_performance['avg_rating'],
            y=category_performance['total_revenue'],
            mode='markers+text',
            text=category_performance['product_category_name'].str.replace('_', ' ').str.title(),
            textposition='top center',
            marker=dict(
                size=category_performance['order_count'] / 20,
                color=category_performance['avg_order_value'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="AOV ($)", x=0.48)
            ),
            name='Categories',
            hovertemplate='Category: %{text}<br>Rating: %{x:.2f}<br>Revenue: $%{y:,.0f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Chart 2: State distribution
    fig.add_trace(
        go.Bar(
            x=state_performance['total_value'],
            y=state_performance['customer_state'],
            orientation='h',
            marker_color=BUSINESS_COLORS['secondary'],
            name='State Revenue',
            hovertemplate='State: %{y}<br>Revenue: $%{x:,.0f}<extra></extra>'
        ),
        row=1, col=2
    )
    
    # Chart 3: Category revenue pie
    fig.add_trace(
        go.Pie(
            labels=category_performance['product_category_name'].str.replace('_', ' ').str.title(),
            values=category_performance['total_revenue'],
            name='Category Revenue',
            hovertemplate='Category: %{label}<br>Revenue: $%{value:,.0f}<br>Percentage: %{percent}<extra></extra>',
            textinfo='label+percent',
            textposition='inside'
        ),
        row=2, col=1
    )
    
    # Chart 4: Delivery performance by state
    fig.add_trace(
        go.Scatter(
            x=state_performance['avg_delivery'],
            y=state_performance['freight_value'],
            mode='markers+text',
            text=state_performance['customer_state'],
            textposition='middle right',
            marker=dict(
                size=state_performance['order_id'] / 100,
                color=BUSINESS_COLORS['info'],
                opacity=0.7
            ),
            name='Delivery Metrics',
            hovertemplate='State: %{text}<br>Delivery Days: %{x:.1f}<br>Avg Freight: $%{y:.2f}<extra></extra>'
        ),
        row=2, col=2
    )
    
    # Chart 5: Weekly patterns
    fig.add_trace(
        go.Bar(
            x=weekly_pattern['weekday'],
            y=weekly_pattern['order_id'],
            marker_color=[
                BUSINESS_COLORS['primary'] if day not in ['Saturday', 'Sunday'] 
                else BUSINESS_COLORS['success'] for day in weekly_pattern['weekday']
            ],
            name='Weekly Orders',
            hovertemplate='Day: %{x}<br>Orders: %{y:,}<extra></extra>'
        ),
        row=3, col=1
    )
    
    # Chart 6: Revenue distribution histogram
    fig.add_trace(
        go.Histogram(
            x=df[df['total_value'] < 500]['total_value'],  # Limit for better visualization
            nbinsx=40,
            marker_color=BUSINESS_COLORS['neutral'],
            opacity=0.7,
            name='Revenue Distribution',
            hovertemplate='Value Range: $%{x}<br>Count: %{y}<extra></extra>'
        ),
        row=3, col=2
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': '⚙️ OPERATIONAL ANALYTICS DASHBOARD',
            'font': {'size': 22, 'color': BUSINESS_COLORS['dark']},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=1000,
        width=1400,
        showlegend=False,
        font=dict(size=10),
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Update specific axes
    fig.update_xaxes(title_text="Average Rating", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=1)
    
    fig.update_xaxes(title_text="Revenue ($)", tickformat='$,.0f', row=1, col=2)
    fig.update_yaxes(title_text="State", row=1, col=2)
    
    fig.update_xaxes(title_text="Delivery Days", row=2, col=2)
    fig.update_yaxes(title_text="Avg Freight ($)", row=2, col=2)
    
    fig.update_xaxes(title_text="Day of Week", row=3, col=1)
    fig.update_yaxes(title_text="Number of Orders", row=3, col=1)
    
    fig.update_xaxes(title_text="Order Value ($)", row=3, col=2)
    fig.update_yaxes(title_text="Frequency", row=3, col=2)
    
    return fig

# Create and display operational dashboard
ops_dashboard = create_operational_dashboard(dashboard_df)
ops_dashboard.show()

print("\n⚙️ Operational Dashboard Features:")
print("  • Detailed performance metrics by category and region")
print("  • Multi-dimensional analysis (rating vs revenue vs volume)")
print("  • Operational efficiency indicators")
print("  • Actionable insights for daily operations")

## 4. Customer Analytics Dashboard

Customer analytics dashboards focus specifically on **customer behavior, satisfaction, and lifecycle metrics**. These are essential for marketing teams, customer success managers, and product teams.

### Key Focus Areas:
- **Customer segmentation** and behavior patterns
- **Satisfaction metrics** and review analysis
- **Purchase patterns** and preferences
- **Geographic distribution** and market penetration

In [6]:
def create_customer_analytics_dashboard(df):
    """
    Create a customer-focused analytics dashboard.
    """
    # Customer segmentation based on order value and frequency
    customer_metrics = df.groupby('customer_id').agg({
        'total_value': ['sum', 'mean', 'count'],
        'review_score': 'mean',
        'order_purchase_timestamp': ['min', 'max']
    }).round(2)
    
    customer_metrics.columns = ['total_spent', 'avg_order_value', 'order_frequency', 
                               'avg_satisfaction', 'first_order', 'last_order']
    customer_metrics = customer_metrics.reset_index()
    
    # Create customer segments
    customer_metrics['customer_lifetime_days'] = (
        customer_metrics['last_order'] - customer_metrics['first_order']
    ).dt.days
    
    # Simple RFM-like segmentation
    customer_metrics['value_segment'] = pd.cut(
        customer_metrics['total_spent'], 
        bins=3, 
        labels=['Low Value', 'Medium Value', 'High Value']
    )
    
    customer_metrics['frequency_segment'] = pd.cut(
        customer_metrics['order_frequency'], 
        bins=3, 
        labels=['Occasional', 'Regular', 'Frequent']
    )
    
    # State-level customer analysis
    state_customers = df.groupby('customer_state').agg({
        'customer_id': 'nunique',
        'total_value': 'sum',
        'review_score': 'mean'
    }).reset_index()
    state_customers['revenue_per_customer'] = state_customers['total_value'] / state_customers['customer_id']
    
    # Monthly customer acquisition
    monthly_acquisition = df.groupby(['year', 'month'])['customer_id'].nunique().reset_index()
    monthly_acquisition['date'] = pd.to_datetime(monthly_acquisition[['year', 'month']].assign(day=1))
    
    # Create customer analytics dashboard
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            '👥 Customer Segmentation (Value vs Frequency)',
            '⭐ Satisfaction Distribution',
            '🗺️ Customer Value by State',
            '📈 Monthly Customer Acquisition',
            '💎 Customer Lifetime Value Distribution',
            '🎯 Segment Performance Matrix'
        ),
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ],
        vertical_spacing=0.08,
        horizontal_spacing=0.1
    )
    
    # Chart 1: Customer segmentation scatter
    colors = {'Low Value': BUSINESS_COLORS['warning'], 'Medium Value': BUSINESS_COLORS['secondary'], 'High Value': BUSINESS_COLORS['success']}
    
    for segment in customer_metrics['value_segment'].unique():
        if pd.notna(segment):
            segment_data = customer_metrics[customer_metrics['value_segment'] == segment]
            fig.add_trace(
                go.Scatter(
                    x=segment_data['order_frequency'],
                    y=segment_data['total_spent'],
                    mode='markers',
                    name=segment,
                    marker=dict(
                        color=colors.get(segment, BUSINESS_COLORS['neutral']),
                        size=8,
                        opacity=0.7
                    ),
                    hovertemplate=f'Segment: {segment}<br>Orders: %{{x}}<br>Total Spent: $%{{y:,.2f}}<extra></extra>'
                ),
                row=1, col=1
            )
    
    # Chart 2: Satisfaction distribution
    satisfaction_counts = df['review_score'].value_counts().sort_index()
    
    fig.add_trace(
        go.Bar(
            x=satisfaction_counts.index,
            y=satisfaction_counts.values,
            marker_color=[
                BUSINESS_COLORS['warning'] if x <= 2 else 
                BUSINESS_COLORS['secondary'] if x == 3 else 
                BUSINESS_COLORS['info'] if x == 4 else 
                BUSINESS_COLORS['success'] for x in satisfaction_counts.index
            ],
            name='Satisfaction',
            hovertemplate='Rating: %{x}<br>Count: %{y:,}<extra></extra>'
        ),
        row=1, col=2
    )
    
    # Chart 3: Revenue per customer by state
    top_states = state_customers.nlargest(10, 'revenue_per_customer')
    
    fig.add_trace(
        go.Bar(
            x=top_states['revenue_per_customer'],
            y=top_states['customer_state'],
            orientation='h',
            marker_color=BUSINESS_COLORS['primary'],
            name='Revenue/Customer',
            hovertemplate='State: %{y}<br>Revenue per Customer: $%{x:,.2f}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Chart 4: Monthly customer acquisition
    fig.add_trace(
        go.Scatter(
            x=monthly_acquisition['date'],
            y=monthly_acquisition['customer_id'],
            mode='lines+markers',
            name='New Customers',
            line=dict(color=BUSINESS_COLORS['success'], width=3),
            fill='tonexty',
            hovertemplate='Date: %{x}<br>New Customers: %{y:,}<extra></extra>'
        ),
        row=2, col=2
    )
    
    # Chart 5: Customer lifetime value distribution
    fig.add_trace(
        go.Histogram(
            x=customer_metrics[customer_metrics['total_spent'] < 1000]['total_spent'],
            nbinsx=30,
            marker_color=BUSINESS_COLORS['info'],
            opacity=0.7,
            name='CLV Distribution',
            hovertemplate='CLV Range: $%{x}<br>Customers: %{y}<extra></extra>'
        ),
        row=3, col=1
    )
    
    # Chart 6: Segment performance matrix
    segment_performance = customer_metrics.groupby(['value_segment', 'frequency_segment']).agg({
        'customer_id': 'count',
        'avg_satisfaction': 'mean'
    }).reset_index()
    
    # Create a heatmap-like visualization
    for i, row in segment_performance.iterrows():
        if pd.notna(row['value_segment']) and pd.notna(row['frequency_segment']):
            fig.add_trace(
                go.Scatter(
                    x=[row['frequency_segment']],
                    y=[row['value_segment']],
                    mode='markers+text',
                    text=[f"{row['customer_id']}<br>customers"],
                    textposition='middle center',
                    marker=dict(
                        size=row['customer_id'] / 20,
                        color=row['avg_satisfaction'],
                        colorscale='RdYlGn',
                        cmin=1,
                        cmax=5,
                        showscale=True,
                        colorbar=dict(title="Avg Rating", x=1.02)
                    ),
                    name=f"{row['value_segment']} + {row['frequency_segment']}",
                    hovertemplate=f'Segment: %{{y}} + %{{x}}<br>Customers: {row["customer_id"]}<br>Avg Satisfaction: {row["avg_satisfaction"]:.2f}<extra></extra>'
                ),
                row=3, col=2
            )
    
    # Update layout
    fig.update_layout(
        title={
            'text': '👥 CUSTOMER ANALYTICS DASHBOARD',
            'font': {'size': 22, 'color': BUSINESS_COLORS['dark']},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=1000,
        width=1400,
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        font=dict(size=10),
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    # Update axes
    fig.update_xaxes(title_text="Order Frequency", row=1, col=1)
    fig.update_yaxes(title_text="Total Spent ($)", tickformat='$,.0f', row=1, col=1)
    
    fig.update_xaxes(title_text="Rating (1-5)", row=1, col=2)
    fig.update_yaxes(title_text="Number of Orders", row=1, col=2)
    
    fig.update_xaxes(title_text="Revenue per Customer ($)", row=2, col=1)
    fig.update_yaxes(title_text="State", row=2, col=1)
    
    fig.update_xaxes(title_text="Month", row=2, col=2)
    fig.update_yaxes(title_text="New Customers", row=2, col=2)
    
    fig.update_xaxes(title_text="Customer Lifetime Value ($)", row=3, col=1)
    fig.update_yaxes(title_text="Number of Customers", row=3, col=1)
    
    fig.update_xaxes(title_text="Purchase Frequency", row=3, col=2)
    fig.update_yaxes(title_text="Value Segment", row=3, col=2)
    
    return fig

# Create and display customer analytics dashboard
customer_dashboard = create_customer_analytics_dashboard(dashboard_df)
customer_dashboard.show()

print("\n👥 Customer Analytics Dashboard Features:")
print("  • Customer segmentation and lifecycle analysis")
print("  • Satisfaction metrics and distribution")
print("  • Geographic customer value analysis")
print("  • Acquisition trends and segment performance")


👥 Customer Analytics Dashboard Features:
  • Customer segmentation and lifecycle analysis
  • Satisfaction metrics and distribution
  • Geographic customer value analysis
  • Acquisition trends and segment performance


## 5. Advanced Layout Techniques

Let's explore some advanced techniques for creating sophisticated dashboard layouts that go beyond basic subplots.

### 5.1 Custom Grid Layouts with Mixed Chart Types

In [ ]:
def create_advanced_layout_dashboard(df):
    """
    Demonstrate advanced layout techniques with mixed chart types and custom positioning.
    """
    # Create a complex layout with different sized panels
    fig = make_subplots(
        rows=4, cols=4,
        subplot_titles=[
            'Revenue Trend', '', 'Order Status', 'Customer Satisfaction',
            '', '', 'Category Performance', 'Delivery Time',
            'Geographic Distribution', '', 'Monthly Patterns', 'AOV Trend',
            '', '', '', 'Key Metrics'
        ],
        specs=[
            [{"colspan": 2}, None, {"type": "pie"}, {"type": "indicator"}],
            [{"colspan": 2, "rowspan": 2}, None, {}, {}],
            [None, None, {}, {}],
            [{"colspan": 3}, None, None, {}]
        ],
        vertical_spacing=0.06,
        horizontal_spacing=0.05
    )
    
    # Large panel: Revenue trend (spans 2 columns)
    monthly_revenue = df.groupby(['year', 'month'])['total_value'].sum().reset_index()
    monthly_revenue['date'] = pd.to_datetime(monthly_revenue[['year', 'month']].assign(day=1))
    
    fig.add_trace(
        go.Scatter(
            x=monthly_revenue['date'],
            y=monthly_revenue['total_value'],
            mode='lines+markers',
            name='Revenue Trend',
            line=dict(color=BUSINESS_COLORS['primary'], width=4),
            fill='tonexty'
        ),
        row=1, col=1
    )
    
    # Order status pie chart
    status_counts = df['order_status'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=status_counts.index,
            values=status_counts.values,
            name='Order Status',
            textinfo='percent',
            showlegend=False
        ),
        row=1, col=3
    )
    
    # Customer satisfaction gauge
    avg_satisfaction = df['review_score'].mean()
    fig.add_trace(
        go.Indicator(
            mode="gauge+number",
            value=avg_satisfaction,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "Satisfaction"},
            gauge={
                'axis': {'range': [None, 5]},
                'bar': {'color': BUSINESS_COLORS['success']},
                'steps': [
                    {'range': [0, 3], 'color': BUSINESS_COLORS['light']},
                    {'range': [3, 4], 'color': BUSINESS_COLORS['secondary']},
                    {'range': [4, 5], 'color': BUSINESS_COLORS['success']}
                ]
            }
        ),
        row=1, col=4
    )
    
    # Large panel: Geographic heatmap (spans 2 columns, 2 rows)
    state_data = df.groupby('customer_state').agg({
        'total_value': 'sum',
        'order_id': 'count'
    }).reset_index()
    
    fig.add_trace(
        go.Bar(
            x=state_data['customer_state'],
            y=state_data['total_value'],
            marker_color=state_data['order_id'],
            marker_colorscale='Blues',
            name='State Revenue'
        ),
        row=2, col=1
    )
    
    # Category performance
    category_data = df.groupby('product_category_name')['total_value'].sum().nlargest(8)
    fig.add_trace(
        go.Bar(
            x=category_data.values,
            y=[cat.replace('_', ' ').title() for cat in category_data.index],
            orientation='h',
            marker_color=BUSINESS_COLORS['secondary'],
            name='Categories'
        ),
        row=2, col=3
    )
    
    # Delivery time distribution
    fig.add_trace(
        go.Histogram(
            x=df[df['delivery_days'] < 30]['delivery_days'],
            nbinsx=15,
            marker_color=BUSINESS_COLORS['info'],
            name='Delivery Days'
        ),
        row=2, col=4
    )
    
    # Monthly order patterns
    monthly_orders = df.groupby('month')['order_id'].count()
    fig.add_trace(
        go.Scatter(
            x=monthly_orders.index,
            y=monthly_orders.values,
            mode='lines+markers',
            line=dict(color=BUSINESS_COLORS['warning'], width=3),
            name='Monthly Orders'
        ),
        row=3, col=3
    )
    
    # AOV trend
    monthly_aov = df.groupby(['year', 'month'])['total_value'].mean().reset_index()
    monthly_aov['date'] = pd.to_datetime(monthly_aov[['year', 'month']].assign(day=1))
    
    fig.add_trace(
        go.Scatter(
            x=monthly_aov['date'],
            y=monthly_aov['total_value'],
            mode='lines+markers',
            line=dict(color=BUSINESS_COLORS['neutral'], width=3),
            name='AOV Trend'
        ),
        row=3, col=4
    )
    
    # Bottom panel: Key metrics summary (spans 3 columns)
    metrics_text = f"""
    <b>📊 KEY BUSINESS METRICS</b><br><br>
    💰 <b>Total Revenue:</b> ${df['total_value'].sum():,.0f}<br>
    📦 <b>Total Orders:</b> {len(df):,}<br>
    👥 <b>Unique Customers:</b> {df['customer_id'].nunique():,}<br>
    💵 <b>Average Order Value:</b> ${df['total_value'].mean():.2f}<br>
    ⭐ <b>Customer Satisfaction:</b> {df['review_score'].mean():.2f}/5.0<br>
    🚚 <b>Avg Delivery Time:</b> {df['delivery_days'].mean():.1f} days
    """
    
    fig.add_annotation(
        text=metrics_text,
        xref="x domain", yref="y domain",
        x=0.1, y=0.5,
        showarrow=False,
        font=dict(size=14, color=BUSINESS_COLORS['dark']),
        align="left",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor=BUSINESS_COLORS['light'],
        borderwidth=2,
        row=4, col=1
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': '🎨 ADVANCED LAYOUT DASHBOARD - MIXED CHART TYPES',
            'font': {'size': 24, 'color': BUSINESS_COLORS['dark']},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=1000,
        width=1400,
        showlegend=False,
        font=dict(size=10),
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa'
    )
    
    return fig

# Create and display advanced layout dashboard
advanced_dashboard = create_advanced_layout_dashboard(dashboard_df)
advanced_dashboard.show()

print("\n🎨 Advanced Layout Features:")
print("  • Mixed chart types in a single dashboard")
print("  • Custom grid layouts with spanning panels")
print("  • Different sized visualization areas")
print("  • Text annotations for key metrics")
print("  • Professional color coordination")

## 6. Dashboard Design Best Practices

### Design Principles for Business Dashboards

1. **Visual Hierarchy**
   - Most important metrics → Top-left position
   - Use size and color to guide attention
   - Group related visualizations together

2. **Color Strategy**
   - Consistent color palette across all charts
   - Semantic colors (red = danger, green = success)
   - Accessible color combinations

3. **Information Density**
   - Balance detail with readability
   - Use progressive disclosure for complex data
   - Prioritize actionable insights

4. **Interactivity**
   - Hover information for context
   - Zoom and pan for exploration
   - Filter capabilities for drill-down

5. **Performance**
   - Optimize for fast loading
   - Sample large datasets appropriately
   - Use efficient chart types

### Dashboard Types Summary

| Dashboard Type | Audience | Focus | Update Frequency |
|---|---|---|---|
| **Executive** | C-Suite, Senior Leadership | High-level KPIs, trends | Weekly/Monthly |
| **Operational** | Managers, Analysts | Detailed metrics, comparisons | Daily/Real-time |
| **Customer Analytics** | Marketing, Product Teams | Customer behavior, satisfaction | Weekly |
| **Financial** | Finance Teams | Revenue, costs, profitability | Monthly/Quarterly |

## 7. Key Takeaways and Next Steps

### What We've Accomplished:

1. **Dashboard Design Principles**
   - Understanding visual hierarchy and information density
   - Professional color strategies for business contexts
   - Layout techniques for different audience needs

2. **Multiple Dashboard Types**
   - Executive dashboards for high-level decision making
   - Operational dashboards for detailed analysis
   - Customer analytics for behavior insights
   - Advanced layouts with mixed chart types

3. **Technical Skills**
   - Complex subplot arrangements
   - Chart coordination and styling
   - Professional presentation techniques
   - Real data integration from zip files

### Business Impact:

- **Improved Decision Making**: Clear, actionable insights at the right level of detail
- **Enhanced Communication**: Visual stories that resonate with different audiences
- **Operational Efficiency**: Self-service analytics reduce manual reporting
- **Strategic Alignment**: Consistent metrics across organizational levels

### Next Session Preview:
In Part 3, we'll explore advanced interactive features including:
- Widget controls and real-time filtering
- Cross-filtering between charts
- Animation and storytelling techniques
- Dashboard deployment strategies

**🎯 You're now equipped to create professional, business-ready dashboards that drive decision-making and deliver value to stakeholders!**

## 8. Practice Challenge

**Your Mission! 🚀**

Create a **Sales Performance Dashboard** that answers these key business questions:

1. *What are our top-performing product categories this quarter?*
2. *How does customer satisfaction vary by state and delivery time?*
3. *What are the seasonal trends in order volume and revenue?*
4. *Which customer segments drive the most value?*

**Requirements:**
- Use a 2x3 subplot layout
- Include at least one gauge/indicator chart
- Apply consistent professional styling
- Add meaningful hover information
- Tell a cohesive business story

**Bonus Points:**
- Add custom annotations with key insights
- Use secondary y-axes for trend comparisons
- Implement color coding for performance levels

In [ ]:
# Your practice challenge code here
# Create a comprehensive Sales Performance Dashboard

def create_sales_performance_dashboard(df):
    """
    Your challenge: Create a sales performance dashboard that tells a business story.
    """
    # Step 1: Prepare your data
    # Step 2: Design your layout (2x3 subplots)
    # Step 3: Create meaningful visualizations
    # Step 4: Apply professional styling
    # Step 5: Add business insights
    
    pass  # Replace with your implementation

print("🎯 Start your Sales Performance Dashboard here!")
print("Think about the story you want to tell with the data...")
print("What insights would be most valuable for business decision-makers?")

# Uncomment and implement:
# sales_dashboard = create_sales_performance_dashboard(dashboard_df)
# sales_dashboard.show()